# OLAP Operationen für GBI Daten
* wir verwenden hier die Daten aus dem BW4 Curriculum
* Dateiformat csv
* getrennte Fakten für US und DE
* Jahre 2007 bis 2019
* Revenue nur in lokaler Währung

Faktentabelle DE einlesen
* Feld- und Dezimaltrenner werden übergeben
* Datum wir geparsed

In [52]:
import pandas as pd

df_facts_de = pd.read_csv("SalesdataBW4DeV01.csv", header = 0,delimiter =';',decimal=',', parse_dates=['SalesDate'])
df_facts_de.head()

,OrderNumber,OrderItem,SalesDate,Customer,Product,SalesQuantity,UnitOfMeasure,Revenue,Currency,Discount,CostsUSD
0,100001,10,2007-01-01,17000,PRTR1000,4,ST,10669.84,EUR,320.10,8320.0
1,100001,20,2007-01-01,17000,DXTR1000,8,ST,20005.92,EUR,600.18,15600.0
2,100001,30,2007-01-01,17000,DXRD2000,2,ST,2750.82,EUR,82.52,2046.0
3,100001,40,2007-01-01,17000,ORWN1000,5,ST,10419.75,EUR,312.59,7500.0
4,100002,10,2007-03-01,15000,PRTR1000,4,ST,10669.84,EUR,533.49,8320.0


Faktentabelle US einlesen 

In [54]:
df_facts_us = pd.read_csv("SalesdataBW4UsV01.csv", header = 0,delimiter =',',decimal='.', parse_dates=['SalesDate'])
df_facts_us.head()


,OrderNumber,OrderItem,SalesDate,Customer,Product,SalesQuantity,UnitOfMeasure,Revenue,Currency,Discount,CostsUSD
0,200001,10,2007-01-01,2000,DXRD1000,7,ST,11900.0,USD,357.0,7378.0
1,200002,10,2007-01-02,5000,PUMP1000,7,ST,196.0,USD,9.8,98.0
2,200002,20,2007-01-02,5000,ORHT2000,1,ST,1700.0,USD,85.0,1020.0
3,200003,10,2007-01-02,11000,ORWN1000,7,ST,17500.0,USD,525.0,10500.0
4,200003,20,2007-01-02,11000,FAID1000,18,ST,720.0,USD,21.6,360.0


Fakten zusammenführen

In [55]:
df_facts = pd.concat([df_facts_de, df_facts_us])
len(df_facts.axes[0])

171010

Dimension Customer einlesen

In [56]:
df_cust = pd.read_csv("DimCustomer.csv", header = 0,delimiter =';')
df_cust.head()

,Customer,CustDescr,City,Salesorg,Country
0,1000,Rocky Mountain Bikes,Denver,UW00,US
1,2000,Big Apple Bikes,New York City,UE00,US
2,3000,Philly Bikes,Philadelphia,UE00,US
3,4000,Peachtree Bikes,Atlanta,UE00,US
4,5000,Beantown Bikes,Boston,UE00,US


Inner Join Faktentabelle und Dimensionstabelle

In [57]:
df_analysis = pd.merge(df_facts, df_cust, how='inner', left_on = 'Customer', right_on = 'Customer')
df_analysis.head()

,OrderNumber,OrderItem,SalesDate,Customer,Product,SalesQuantity,UnitOfMeasure,Revenue,Currency,Discount,CostsUSD,CustDescr,City,Salesorg,Country
0,100001,10,2007-01-01,17000,PRTR1000,4,ST,10669.84,EUR,320.10,8320.0,Cruiser Bikes,Hannover,DN00,DE
1,100001,20,2007-01-01,17000,DXTR1000,8,ST,20005.92,EUR,600.18,15600.0,Cruiser Bikes,Hannover,DN00,DE
2,100001,30,2007-01-01,17000,DXRD2000,2,ST,2750.82,EUR,82.52,2046.0,Cruiser Bikes,Hannover,DN00,DE
3,100001,40,2007-01-01,17000,ORWN1000,5,ST,10419.75,EUR,312.59,7500.0,Cruiser Bikes,Hannover,DN00,DE
4,100005,10,2007-04-01,17000,DXTR2000,8,ST,20005.92,EUR,600.18,15600.0,Cruiser Bikes,Hannover,DN00,DE


Auswertung mit Aggregation eines Feldes

In [49]:
import numpy as np
df_analysis.groupby('Country').agg({'SalesQuantity': np.sum, 'Revenue': np.sum})

,SalesQuantity,Revenue
Country,,
DE,477545,7.633314e+08
US,426201,7.580509e+08


Jahr ableiten

In [61]:
df_analysis['Year'] =  df_analysis['SalesDate'].dt.year
df_analysis.groupby('Year').agg({'SalesQuantity': np.sum})

,SalesQuantity
Year,
2007,71542
2008,68063
2009,58838
2010,65049
2011,73579
2012,73168
2013,72035
2014,70540
2015,72896


Aggregation mehrerer Felder; kann mit den Zahlen in Case A2 verglichen werden

In [66]:
df_analysis.groupby(['Year', 'Country']).agg({ 'Revenue': np.sum})

Revenue
Year Country              
2007 DE       4.399617e+07
     US       5.615485e+07
2008 DE       4.119568e+07
     US       5.620023e+07
2009 DE       4.454352e+07
     US       3.961103e+07
2010 DE       4.901596e+07
     US       4.617362e+07
2011 DE       5.357778e+07
     US       6.093752e+07
2012 DE       5.574937e+07
     US       6.702040e+07
2013 DE       5.552064e+07
     US       6.300935e+07
2014 DE       5.874238e+07
     US       6.336271e+07
2015 DE       6.728483e+07
     US       5.716730e+07
2016 DE       7.005875e+07
     US       5.603304e+07
2017 DE       7.752310e+07
     US       6.501220e+07
2018 DE       7.087010e+07
     US       6.157447e+07
2019 DE       7.525314e+07
     US       6.579420e+07

Filter setzen

In [67]:
df_analysis[df_analysis['Country']=='DE'].groupby('Year').agg({'Revenue': np.sum})

,Revenue
Year,
2007,4.399617e+07
2008,4.119568e+07
2009,4.454352e+07
2010,4.901596e+07
2011,5.357778e+07
2012,5.574937e+07
2013,5.552064e+07
2014,5.874238e+07
2015,6.728483e+07


Drill Down, usw...

In [69]:
df_analysis[df_analysis['Country']=='DE'].groupby(['Year','Salesorg']).agg({'Revenue': np.sum})

Revenue
Year Salesorg             
2007 DN00      25251754.17
     DS00      18744412.54
2008 DN00      21923105.37
     DS00      19272571.49
2009 DN00      23572780.00
     DS00      20970742.64
2010 DN00      26360641.69
     DS00      22655321.27
2011 DN00      28264822.26
     DS00      25312958.74
2012 DN00      30579786.06
     DS00      25169588.07
2013 DN00      30126661.23
     DS00      25393974.76
2014 DN00      32252905.18
     DS00      26489478.73
2015 DN00      35040308.29
     DS00      32244523.89
2016 DN00      36858641.63
     DS00      33200103.61
2017 DN00      41403417.73
     DS00      36119679.37
2018 DN00      39729049.69
     DS00      31141048.37
2019 DN00      42387674.55
     DS00      32865467.17